# Structure of ViT model

**Objective:** Understand the structure of the ViT we are working with.

# Importing libraries

In [7]:
import torch
from PIL import Image
import numpy
import sys
from torchvision import transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt

from vit_rollout import VITAttentionRollout
from vit_grad_rollout import VITAttentionGradRollout

# Loading model

In [11]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE}")

model = torch.hub.load('facebookresearch/deit:main', 
        'deit_tiny_patch16_224', pretrained=True)
model.eval()
model.to(DEVICE)

Using cpu


Using cache found in /users/eleves-a/2018/nicolas.lopes/.cache/torch/hub/facebookresearch_deit_main


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=768, out_features=192, bias=True)
        (drop2): Dropout(p=0.0, inplace=Fals

# Model Structure

In short, the model has three main structures:

- Tokenization + positional embedding
- 12 Transformer Layers
- One fully connected layer for classification

## Tokenization + positional embedding

The model starts by performing an "tokezination" + positional embedding of our image: (check which part adds the position embedding)

In [10]:
'''
(patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
'''

'\n(patch_embed): PatchEmbed(\n    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))\n    (norm): Identity()\n  )\n'

## Transformer Layers

Basically the model is a sequence of **12** transformer blocks that are **the same**. Here is a structure of a transformer head:

Obs: Most likely, the (attn_drop) with p = 0 has only the intention of getting the attention layer.

In [6]:
'''
(8): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=768, out_features=192, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (ls2): Identity()
      (drop_path2): Identity()
    )
'''

"\n(8): Block(\n      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)\n      (attn): Attention(\n        (qkv): Linear(in_features=192, out_features=576, bias=True)\n        (attn_drop): Dropout(p=0.0, inplace=False)\n        (proj): Linear(in_features=192, out_features=192, bias=True)\n        (proj_drop): Dropout(p=0.0, inplace=False)\n      )\n      (ls1): Identity()\n      (drop_path1): Identity()\n      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)\n      (mlp): Mlp(\n        (fc1): Linear(in_features=192, out_features=768, bias=True)\n        (act): GELU(approximate='none')\n        (drop1): Dropout(p=0.0, inplace=False)\n        (fc2): Linear(in_features=768, out_features=192, bias=True)\n        (drop2): Dropout(p=0.0, inplace=False)\n      )\n      (ls2): Identity()\n      (drop_path2): Identity()\n    )\n"

## Fully connected layer for classification

Finally at the end we have only a fully connected layer for classification

In [ ]:
'''
(norm): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
  (fc_norm): Identity()
  (head): Linear(in_features=192, out_features=1000, bias=True)
'''